In [ ]:
import os

from bids import BIDSLayout
from nipype.pipeline import engine as pe
from nipype.interfaces import mrtrix3, utility as niu

In [ ]:
def init_dwi_artifacts_wf(ignore, output_resolution):
    
    wf = pe.Workflow(name='dwi_artifacts_wf')

    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_file']), name='inputnode')

    outputnode = pe.Node(niu.IdentityInterface(fields=['out_file']), name='outputnode')

    dwibuffer = pe.Node(niu.IdentityInterface(fields=['dwi_file']), name='dwibuffer')

    denoise = pe.Node(mrtrix3.DWIDenoise(), name='denoise')

    unring = pe.Node(mrtrix3.MRDeGibbs(), name='unring')

    resize = pe.Node(mrtrix3.MRResize(), name='resize')

    if ignore == ['denoising']:
        wf.connect([
            (inputnode, unring, [('dwi_file', 'in_file')]),
            (unring, dwibuffer, [('out_file', 'dwi_file')])
        ])

    elif ignore == ['unringing']:
        wf.connect([
            (inputnode, denoise, [('dwi_file', 'in_file')]),
            (denoise, dwibuffer, [('out_file', 'dwi_file')])
        ])

    elif ['denoising', 'unringing'] in ignore:
        wf.connect([
            (inputnode, dwibuffer, 'dwi_file', 'dwi_file')
        ])

    else:
        wf.connect([
            (inputnode, denoise, [('dwi_file', 'in_file')]),
            (denoise, unring, [('out_file', 'in_file')]),
            (unring, dwibuffer, [('out_file', 'dwi_file')])
        ])

    if output_resolution:
        resize.inputs.voxel_size=output_resolution
        wf.connect([
            (dwibuffer, resize, [('dwi_file', 'in_file')]),
            (resize, outputnode, [('out_file', 'out_file')])
        ])

    else:
        wf.connect([
            (dwibuffer, outputnode, [('dwi_file', 'out_file')])
        ])

    return wf

In [ ]:
data_dir = os.path.abspath("../data")
layout = BIDSLayout(data_dir)

In [ ]:
dwi_files = layout.get(datatype="dwi", extension=["nii.gz", "nii"], return_type="file")

In [ ]:
dwi_file = dwi_files[-5]
ignore = ['denoising']
output_resolution = ()

In [ ]:
test_wf = init_dwi_artifacts_wf(ignore, output_resolution)
test_wf.base_dir = os.getcwd()

inputspec = test_wf.get_node("inputnode")
inputspec.inputs.dwi_file = dwi_file

test_wf.write_graph(graph2use="colored")
test_wf.config["execution"]["remove_unnecessary_outputs"] = False
test_wf.config["execution"]["keep_inputs"] = True
test_wf.config["execution"]["crashfile_format"] = "txt"

test_wf.run()